In [ ]:
import numpy as np
import pandas as pd
import random
import math
import statsmodels.formula.api as smf
import sys
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier,AdaBoostRegressor
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from mlxtend.regressor import StackingCVRegressor,StackingRegressor
from sklearn.neural_network import MLPClassifier,MLPRegressor
import itertools
from scipy import stats
from scipy.special import expit
xgb.set_config(verbosity=0)

In [ ]:
'''
In real data analysis, we no longer need to generate sample image, 
    but only need to compute the eigenscores of the MRI image.
'''
from __future__ import division
from scipy.optimize import fsolve, root
from scipy import integrate
from sympy import Symbol, exp, log
from numpy import linalg as la
import matplotlib.pyplot as plt
import numpy as np
import time
import scipy.stats
import numpy.random as nrd
import multiprocessing

def get_est_U(df,l=300):
    df['idx'] = range(len(df))
    df = df.set_index('idx')
    N = len(df.index)
    pg = 601
    L = l
    X_bar_part_lst = []
    X_bar_square = np.zeros(shape=[N, N])
    part_img = np.zeros(shape=[N,L,pg])
    for i in range(N):
        img_path = './triple-crossfitting/total_image_array/'+df['PTID'][i]+'-img_part.npy'
        img = np.load(img_path)
        part_img[i] = img
    aa = part_img.reshape((N,-1))/601
    aa = aa - np.mean(aa, 0)[np.newaxis] 
    bb = aa.reshape((N,L,pg))
    for l in range(L):
        X_bar_part = bb[:,l,:]
        X_bar_square += np.matmul(X_bar_part, X_bar_part.transpose())
        X_bar_part_lst.append(X_bar_part)
    v, s_2, vt = la.svd(X_bar_square)  # u: n*n, s: n (eigenvalue)
    est_eimg = np.zeros([N, L, pg])

    for l in range(L):
        X_bar_part = X_bar_part_lst[l]
        est_eimg[:, l] = np.matmul(X_bar_part.T, np.matmul(v, np.sqrt(np.diag(1/s_2)))).T # U(m)=X(m)VS_-1
    U = est_eimg.reshape((N, -1))
    return U#est_eimg

def get_eigenscore(df,est_u,ll=300):
    N = len(df.index)
    L = ll
    grid = 601
    part_img = np.zeros(shape=[N,L, grid])
    for i in range(N):
        img_path = './triple-crossfitting/total_image_array/'+df['PTID'][i]+'-img_part.npy'
        img = np.load(img_path)
        part_img[i] = img
    aa = part_img.reshape((N,-1))/601
    aa = aa - np.mean(aa, 0)[np.newaxis] 
    eigenscore = np.matmul(est_u,aa.T)
    return eigenscore

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from pygam import GAM,LogisticGAM
class DRModel():
    def __init__(self):
        self.binary_model_lst = None
        self.continue_model_lst = None
        self.train_df = None
        self.test_df = None 
    def data_loadin(self,train_df,test_df):
        self.train_df = train_df
        self.test_df = test_df 
    def ml_models(self,binary_models,continue_models):
        self.binary_model_lst = binary_models
        self.continue_model_lst = continue_models
        
    def KFold_val(self,train=None,indep_var=None,dep_var=None,model=None,model_type = 'binary'):
        X, Y = pd.DataFrame(train[indep_var]), pd.DataFrame(train[dep_var])
        if model_type == 'binary':
            scores = cross_val_score(model, X, Y, cv=3, scoring='accuracy')
        elif model_type == 'continue':
            scores = cross_val_score(model, X, Y, cv=3, scoring='neg_mean_squared_error')
        return np.mean(scores)      
    
    def stacking(self,formula,train=None,test=None,return_proba=False,model_type = 'binary'):
        indep_var = formula.split('~')[-1].replace(' ','').split('+')
        dep_var = formula.split('~')[0].replace(' ','')
        stacking_df = pd.DataFrame()
        stacking_test_df = pd.DataFrame()
        if model_type == 'binary':
            model_lst = self.binary_model_lst
        elif model_type == 'continue':
            model_lst = self.continue_model_lst 
        for model in model_lst:
            result = np.array([])
            for train_idx,text_idx in KFold(n_splits=3).split(train):
                model1 = model
                X_train, X_test = train.loc[train_idx,indep_var], train.loc[text_idx,indep_var]
                y_train, y_test = train.loc[train_idx,dep_var], train.loc[text_idx,dep_var]
                result = np.append(result,model1.fit(X_train,y_train).predict(X_test))
            stacking_df[str(model).split('(')[0]] = result
            model.fit(train[indep_var],train[dep_var])
            stacking_test_df[str(model).split('(')[0]] = model.predict(test[indep_var])
        stacking_df['label'] = train[dep_var]
        
        if model_type == 'binary':
            meta_model = LogisticRegression()
            meta_model.fit(stacking_df[stacking_df.columns[:-1]],stacking_df['label'])
            if return_proba==True:
                res = meta_model.predict_proba(stacking_test_df)[:,1]
            else:
                res = meta_model.predict(stacking_test_df)
        elif model_type == 'continue':
            meta_model = LinearRegression()
            meta_model.fit(stacking_df[stacking_df.columns[:-1]],stacking_df['label'])
            res = meta_model.predict(stacking_test_df)
        return res
            
        
    def ps_Model(self,formula,ml_method=False,model_type='binary'):
        indep_var = formula.split('~')[-1].replace(' ','').split('+')
        treatment_var = formula.split('~')[0].replace(' ','')
        # calculate propensity score
        start = time.perf_counter()
        if ml_method==True:
            if model_type=='binary':
                mods = self.binary_model_lst.copy()[1:]
            elif model_type == 'continue':
                mods = self.continue_model_lst.copy()[1:]
            ml_val_lst = []
            for model1 in mods:
                ml_val_lst.append(dr.KFold_val(self.train_df,indep_var,treatment_var,model1))
            model = mods[np.argmax(ml_val_lst)]
            model.fit(self.train_df[indep_var],self.train_df[treatment_var])
            try:
                try:
                    propensity_score = model.predict_proba(self.test_df[indep_var])[:,1]
                except:
                    propensity_score = model.predict_proba(self.test_df[indep_var])
            except:
                propensity_score = model.predict(self.test_df[indep_var])[:,1]
            propensity_score = np.where(propensity_score<0.005,0.005,propensity_score)
            propensity_score = np.where(propensity_score>0.995,0.995,propensity_score)
        else:
            if model_type=='binary':
                model = LogisticRegression()
            elif model_type == 'continue':
                model = LinearRegression()
            model.fit(self.train_df[indep_var],self.train_df[treatment_var])
            propensity_score = model.predict_proba(self.test_df[indep_var])[:,1]
        return propensity_score,time.perf_counter()-start 
            
    def rsp_Model(self,formula,label,ml_method=False,model_type='binary'):
        indep_var = formula.split('~')[-1].replace(' ','').split('+')
        response_var = formula.split('~')[0].replace(' ','')
        value_y = label
        train = self.train_df[self.train_df['APOE4']==int(value_y)]
        train.reset_index(drop=True,inplace=True)
        start = time.perf_counter()
        if ml_method==True:
            if model_type=='binary':
                mods = self.binary_model_lst.copy()[1:]
                stacking_model = StackingClassifier(self.binary_model_lst[1:],LG_psmodel)
            elif model_type == 'continue':
                mods = self.continue_model_lst.copy()[1:]
                stacking_model = StackingRegressor(self.continue_model_lst[1:],Linear_rspmodel)            
            mods.append(stacking_model)
            ml_val_lst = []
            for model1 in mods:
                ml_val_lst.append(dr.KFold_val(train,indep_var,response_var,model1,model_type=model_type))
            model = mods[np.argmax(ml_val_lst)]
        else:
            if model_type=='binary':
                model = LogisticRegression()
            elif model_type == 'continue':
                model = LinearRegression()
        model.fit(train[indep_var],train[response_var])
        ui_x = model.predict(self.test_df[indep_var])       
        return ui_x,time.perf_counter()-start 
    
    
LG_psmodel = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000)
Linear_rspmodel = LinearRegression()

RF_psmodel = RandomForestClassifier()
RF_rspmodel = RandomForestRegressor()

ADA_rspmodel = AdaBoostRegressor(learning_rate=0.05)
ADA_psmodel = AdaBoostClassifier()

XGB_rspmodel = xgb.XGBRegressor(n_estimators=500,learning_rate=0.005)
XGB_psmodel = xgb.XGBClassifier(n_estimators=500,learning_rate=0.005)

GAM_rspmodel = GAM(link='identity',n_splines=4,lam=0.6,max_iter=10000)
GAM_psmodel =  LogisticGAM(n_splines=4,lam=0.6,max_iter=10000)

BPNN_rspmodel = MLPRegressor(hidden_layer_sizes=(128,),activation='relu',max_iter=100,solver='lbfgs')
BPNN_psmodel = MLPClassifier(hidden_layer_sizes=(128,),activation='relu',max_iter=100,solver='lbfgs')

ps_mds = [LG_psmodel,RF_psmodel,ADA_psmodel,XGB_psmodel,BPNN_psmodel]
rsp_mds = [Linear_rspmodel,RF_rspmodel,ADA_rspmodel,GAM_rspmodel]

dr = DRModel()
dr.ml_models(binary_models=ps_mds,continue_models=rsp_mds)
ev_keys = ['ML']

In [ ]:
# other functions
def flatten(lis):
    ret=[]
    for item in lis:
        if not isinstance(item,list):
            ret.append(item)
        else:
            ret.extend(flatten(item))
    return ret
def coverage(p,lower,upper,true_value):
    pp=len(p)
    count = 0.0
    for _ in range(pp):
        if true_value[_] >= lower[_] and true_value[_] <= upper[_]:
            count+=1.0
    return count/pp

def I(val_lst,min_thred=-100000,max_thred=100000):
    kk = []
    for val in val_lst:
        if val>=min_thred and val<=max_thred:
            kk.append(1)
        else:
            kk.append(0)
    return np.array(kk)

def frame_reindex(frame):
    a = pd.DataFrame(frame,columns = ['egs'])
    a['idx'] = ['es'+str(i) for i in range(len(a))]
    a = a.set_index('idx')
    return pd.Series(a['egs'])
def sample_splitting(dataframe,resample=False):
    simu_frame_s = dataframe    
    frame_length = len(simu_frame_s.index)
    
    simu_frame1 = simu_frame_s.iloc[:int(frame_length*0.25),:]
    image_observer1 = simu_frame1['PTID']
    
    simu_frame2 = simu_frame_s.iloc[int(frame_length*0.25):int(frame_length*0.5),:]
    image_observer2 = simu_frame2['PTID']
    
    simu_frame3 = simu_frame_s.iloc[int(frame_length*0.5):int(frame_length*0.75),:]
    image_observer3 = simu_frame3['PTID']
    
    simu_frame4 = simu_frame_s.iloc[int(frame_length*0.75):,:]
    image_observer4 = simu_frame4['PTID']
    
    return simu_frame1,image_observer1,simu_frame2,image_observer2,simu_frame3,image_observer3,simu_frame4,image_observer4   


def dc_sample_splitting(simu_frame_s):
    frame_length = len(simu_frame_s.index)
    simu_frame1 = simu_frame_s.iloc[:int(frame_length*0.333333),:]
    simu_frame2 = simu_frame_s.iloc[int(frame_length*0.333333):int(frame_length*0.666666),:]
    simu_frame3 = simu_frame_s.iloc[int(frame_length*0.666666):,:]
    
    return simu_frame1,simu_frame2,simu_frame3      

In [ ]:
# loadin the processed data without eigenscores
df = pd.read_csv('./triple-crossfitting/processed_df.csv')
df = df.drop('idx',axis=1)

In [ ]:
# calculate eigenscores with all data for use in computing "baseline DR" and "DC-DR"
est_u = get_est_U(df,l=300)
eigenframe = pd.DataFrame(get_eigenscore(df,est_u,ll=300))
CPV_frame = ((eigenframe.var())/((eigenframe.var()).sum())).sort_values(ascending = False).cumsum()
eigenframe = eigenframe.T.reindex(CPV_frame.index).T
CPV_frame = frame_reindex(CPV_frame)
eigenframe.columns = CPV_frame.index
choose_index = CPV_frame[CPV_frame<0.95].index
t_simu_frame = df.merge(eigenframe[[i for i in choose_index]],left_index=True,right_index=True)
t_simu_frame.to_csv('./triple-crossfitting/t_simu_frame.csv')


In [ ]:
# loadin the processed data with eigenscores
t_simu_frame = pd.read_csv('./triple-crossfitting/t_simu_frame.csv')
colu = t_simu_frame1.columns[1:]
general_formu = colu.drop(['ADAS11', 'ADAS13', 'MMSE','APOE4'])
dep_indicator = 'ADAS13' #ADAS11 MMSE
ps_formula = 'APOE4 ~ '+'+'.join(list(general_formu))

In [ ]:
# calculate baseline ATE using machine learning method
baseline_ate = {}
baseline_time_lst = {}
baseline_ate['G_Computation'] = {}
baseline_ate['IPW'] = {}
baseline_ate['DR'] = {}
baseline_time_lst['G_Computation'] = {}
baseline_time_lst['IPW'] = {}
baseline_time_lst['DR'] = {}
true_ate = []
part1_time = 0
ps_tm = {}
rsp_tm = {}
ev_keys = ['ML']
for k in ev_keys:
    baseline_ate['G_Computation'][k] = {}
    baseline_ate['IPW'][k] = {}
    baseline_ate['DR'][k] = {}
    ps_tm[k] = 0
    rsp_tm[k] = 0
for __ in partition:
    for k in ev_keys:
        baseline_ate['G_Computation'][k][__] = []
        baseline_ate['IPW'][k][__] = []
        baseline_ate['DR'][k][__] = []  
    t_simu_frame = t_simu_frame1.sample(frac=1)
    t_simu_frame = t_simu_frame1.iloc[:1350,:]
    t_simu_frame['idx'] = range(len(t_simu_frame))
    t_simu_frame = t_simu_frame.set_index('idx')
    frame_length = len(t_simu_frame)
    t_simu_frame['par'] = flatten([i]*675 for i in range(2))
    for cv in range(2):
        train_simu_frame = t_simu_frame[t_simu_frame['par']!=cv]   
        test_simu_frame = t_simu_frame[t_simu_frame['par']==cv]      

        train_simu_frame['idx'] = range(len(train_simu_frame.index))
        train_simu_frame.set_index('idx',inplace=True)
        test_simu_frame['idx'] = range(len(test_simu_frame.index))
        test_simu_frame.set_index('idx',inplace=True)

        dr.data_loadin(train_simu_frame,test_simu_frame)
        ps_formula = 'APOE4 ~ '+'+'.join(list(general_formu))
        rsp_formula = [dep_indicator, '~ '+'+'.join(list(general_formu)),
                       '~ '+'+'.join(list(general_formu))]
        for sec_keys in ev_keys:
            if sec_keys=='ML':
                ml_flag=True
                test_simu_frame['propensity_score'],ps_time = dr.ps_Model(ps_formula,ml_method=ml_flag) 
                ps_tm[sec_keys]+=ps_time
                rsp_formu = rsp_formula[0]+rsp_formula[1]
                test_simu_frame['u0_X'],rsp_t = dr.rsp_Model(rsp_formu,0,ml_method=ml_flag,model_type='continue')     
                rsp_formu = rsp_formula[0]+rsp_formula[2]
                test_simu_frame['u1_X'],rsp_t = dr.rsp_Model(rsp_formu,1,ml_method=ml_flag,model_type='continue')     
                rsp_tm[sec_keys]+=rsp_t
            else:
                ml_flag=False
                test_simu_frame['propensity_score'],ps_time = dr.ps_Model(ps_formula,ml_method=ml_flag) 
                ps_tm[sec_keys]+=ps_time
                response_time = 0
                for label in [0,1]:
                    rsp_formu = rsp_formula[0]+rsp_formula[label+1]
                    test_simu_frame['u'+str(label)+'_X'],rsp_t = dr.rsp_Model(rsp_formu,label,ml_method=False,model_type='continue')     
                    rsp_tm[sec_keys]+=rsp_t
            baseline_data = test_simu_frame[['APOE4','propensity_score','u1_X','u0_X']] 
            baseline_data['Y_F'] = test_simu_frame[dep_indicator]
            baseline_ate_pos = (baseline_data['APOE4']*(baseline_data['Y_F']-baseline_data['u1_X']))/baseline_data['propensity_score'] + baseline_data['u1_X']
            baseline_ate_neg = ((1.0-baseline_data['APOE4'])*(baseline_data['Y_F']-baseline_data['u0_X']))/(1.0-baseline_data['propensity_score']) + baseline_data['u0_X']           
            true_ate.append(baseline_data[baseline_data['APOE4']==1]['Y_F'].mean()-baseline_data[baseline_data['APOE4']==0]['Y_F'].mean())

            DR = baseline_ate_pos-baseline_ate_neg
            G_Computation = baseline_data['u1_X'] - baseline_data['u0_X']
            IPW = ((baseline_data['APOE4']*baseline_data['Y_F'])/baseline_data['propensity_score'])-(((1.0-baseline_data['APOE4'])*baseline_data['Y_F'])/(1.0-baseline_data['propensity_score']))

            baseline_ate['G_Computation'][sec_keys][__].append(G_Computation)
            baseline_ate['IPW'][sec_keys][__].append(IPW)
            baseline_ate['DR'][sec_keys][__].append(DR)
np.save('./triple-crossfitting/true/ADAS13baseline_ate.npy',baseline_ate)

In [ ]:
# Real data Run-time
print (('IPW-ML:')+str((part1_time+ps_tm['ML'])))
print (('G-ML:')+str((part1_time+rsp_tm['ML'])))
print (('DR-ML:')+str((part1_time+rsp_tm['ML']+ps_tm['ML'])))

In [ ]:
# calculate DC-DR using machine learning method
dc_ate = {}
dc_time_lst = {}
dc_ate['DC_DR'] = {}
dc_time_lst['DC_DR'] = {}
splits = [1,2,3]
part1_time = 0
ps_tm = {}
rsp_tm = {}
ev_keys = ['ML']
true_ate = []
for k in ev_keys:
    dc_ate['DC_DR'][k] = {}
    dc_time_lst['DC_DR'][k] = []

ps_formula = 'APOE4 ~ '+'+'.join(list(general_formu))
rsp_formula = [dep_indicator, '~ '+'+'.join(list(general_formu)),
               '~ '+'+'.join(list(general_formu))]
for __ in partition:
    t_simu_frame = t_simu_frame1.sample(frac=1)
    t_simu_frame = t_simu_frame.iloc[:1350,:] # to ensure the frame can be divided into three equal splits
    start = time.perf_counter()
    t_simu_frame['idx'] = range(len(t_simu_frame))
    t_simu_frame = t_simu_frame.set_index('idx')
    sys.stdout.write('\r'+str(__+1)+'/100')
    # cross-modeling
    simu_frame1,simu_frame2,simu_frame3 = dc_sample_splitting(t_simu_frame)
    # hdfpca / psm-model / response
    for o in splits:
        eval('simu_frame'+str(o))['idx'] = range(len(eval('simu_frame'+str(o)).index))
        eval('simu_frame'+str(o)).set_index('idx',inplace=True)
    for kys in ev_keys:
        dc_ate['DC_DR'][kys][__] = []
        ps_tm[kys] = 0
        rsp_tm[kys] = 0
    combi_lst = (list(itertools.permutations(splits, 3)))

    for combi in combi_lst: #【2,3,4】
        simu_frame11 = simu_frame1.copy()
        simu_frame12 = simu_frame2.copy()
        simu_frame13 = simu_frame3.copy()
        part1_time+=time.perf_counter()-start
        # propensity score        
        for sec_keys in ev_keys:    
            if sec_keys=='ML':
                ml_flag=True
                dr.data_loadin(eval('simu_frame1'+str(combi[1])),eval('simu_frame1'+str(combi[0])))  
                eval('simu_frame1'+str(combi[0]))['propensity_score'],ps_time = dr.ps_Model(ps_formula,ml_flag) 
                ps_tm[sec_keys]+=ps_time
                dr.data_loadin(eval('simu_frame1'+str(combi[2])),eval('simu_frame1'+str(combi[0])))
                rsp_formu = rsp_formula[0]+rsp_formula[1]
                eval('simu_frame1'+str(combi[0]))['u0_X'],rsp_t = dr.rsp_Model(rsp_formu,0,ml_flag,model_type='continue')   
                rsp_tm[sec_keys]+=rsp_t
                rsp_formu = rsp_formula[0]+rsp_formula[2]
                eval('simu_frame1'+str(combi[0]))['u1_X'],rsp_t = dr.rsp_Model(rsp_formu,1,ml_flag,model_type='continue')   
                rsp_tm[sec_keys]+=rsp_t
            else:
                ml_flag=False
                dr.data_loadin(eval('simu_frame1'+str(combi[1])),eval('simu_frame1'+str(combi[0])))
                eval('simu_frame1'+str(combi[0]))['propensity_score'],ps_time = dr.ps_Model(ps_formula,ml_flag) 
                ps_tm[sec_keys]+=ps_time
                dr.data_loadin(eval('simu_frame1'+str(combi[2])),eval('simu_frame1'+str(combi[0])))
                for label in [0,1]:
                    rsp_formu = rsp_formula[0]+rsp_formula[label+1]
                    eval('simu_frame1'+str(combi[0]))['u'+str(label)+'_X'],response_time = dr.rsp_Model(rsp_formu,label,ml_flag,model_type='continue')   
                    rsp_tm[sec_keys]+=rsp_t

            data = eval('simu_frame1'+str(combi[0]))[['APOE4','propensity_score','u1_X','u0_X']] 
            data['Y_F'] = eval('simu_frame1'+str(combi[0]))[dep_indicator]
            ate_pos = (data['APOE4']*(data['Y_F']-data['u1_X']))/data['propensity_score'] + data['u1_X']
            ate_neg = ((1.0-data['APOE4'])*(data['Y_F']-data['u0_X']))/(1.0-data['propensity_score']) + data['u0_X']           
            DC_DR = ate_pos-ate_neg
            true_ate.append(data[data['APOE4']==1]['Y_F'].mean()-data[data['APOE4']==0]['Y_F'].mean())

            dc_ate['DC_DR'][sec_keys][__].append(DC_DR)
    sys.stdout.flush()
np.save('./triple-crossfitting/true/ADAS13dc_ate.npy',dc_ate)

In [ ]:
# Run-time
str((part1_time+rsp_tm['ML']+ps_tm['ML']))

In [ ]:
# calculate TC-DR using machine learning method
import warnings
warnings.filterwarnings('ignore')
# main simulation section
splits = [1,2,3,4]
true_ate = []
part1_time = 0
ps_tm = {}
rsp_tm = {}
partition_ate = {}
partition_time_lst = {}
partition_ate['TC_DR'] = {}
partition_time_lst['TC_DR'] = {}
for k in ev_keys:
    partition_ate['TC_DR'][k] = {}
    partition_time_lst['TC_DR'][k] = []
for __ in partition:
    sys.stdout.write('\r'+str(__+1)+'/100')
    # cross-modeling
    p_simu_frame = df.sample(frac=1)
    p_simu_frame = p_simu_frame.iloc[:1348,:] # to ensure the frame can be divided into four equal splits
    p_simu_frame = p_simu_frame.fillna(method='ffill')
    p_simu_frame['idx'] = range(len(p_simu_frame))
    p_simu_frame = p_simu_frame.set_index('idx')
    simu_frame1,image_observer1,simu_frame2,image_observer2,simu_frame3,image_observer3,simu_frame4,image_observer4 = sample_splitting(p_simu_frame,resample=False)
    # hdfpca / psm-model / response    
    for o in splits:
        eval('simu_frame'+str(o))['idx'] = range(len(simu_frame1.index))
        eval('simu_frame'+str(o)).set_index('idx',inplace=True)
    for kys in ev_keys:
        partition_ate['TC_DR'][kys][__] = []
        ps_tm[kys] = 0
        rsp_tm[kys] = 0
    for _ in splits: 
        start = time.perf_counter()
        # calculate U ( X = USV(-1))
        est_u = get_est_U(eval('simu_frame'+str(_)),l=300)
        sec_splits = splits.copy()
        sec_splits.remove(_)
        combi_lst = (list(itertools.permutations(sec_splits, 3)))
        
        eigenframe1 = pd.DataFrame(get_eigenscore(simu_frame1,est_u,ll=300)).T  
        CPV_frame = ((eigenframe1.var())/((eigenframe1.var()).sum())).sort_values(ascending = False).cumsum()
        eigenframe1 = eigenframe1.T.reindex(CPV_frame.index).T
        CPV_frame1 = frame_reindex(CPV_frame)    
        eigenframe1.columns = CPV_frame1.index

        eigenframe2 = pd.DataFrame(get_eigenscore(simu_frame2,est_u,ll=300)).T  
        CPV_frame = ((eigenframe2.var())/((eigenframe2.var()).sum())).sort_values(ascending = False).cumsum()
        eigenframe2 = eigenframe2.T.reindex(CPV_frame.index).T
        CPV_frame2 = frame_reindex(CPV_frame) 
        eigenframe2.columns = CPV_frame2.index

        eigenframe3 = pd.DataFrame(get_eigenscore(simu_frame3,est_u,ll=300)).T
        CPV_frame = ((eigenframe3.var())/((eigenframe3.var()).sum())).sort_values(ascending = False).cumsum()
        eigenframe3 = eigenframe3.T.reindex(CPV_frame.index).T
        CPV_frame3 = frame_reindex(CPV_frame) 
        eigenframe3.columns = CPV_frame3.index
        
        eigenframe4 = pd.DataFrame(get_eigenscore(simu_frame4,est_u,ll=300)).T
        CPV_frame = ((eigenframe4.var())/((eigenframe4.var()).sum())).sort_values(ascending = False).cumsum()
        eigenframe4 = eigenframe4.T.reindex(CPV_frame.index).T
        CPV_frame4 = frame_reindex(CPV_frame) 
        eigenframe4.columns = CPV_frame4.index
        
        
        choose_index = eval('CPV_frame'+str(_))[eval('CPV_frame'+str(_))<0.95].index
        simu_frame1[[i for i in choose_index]] = eigenframe1[[i for i in choose_index]]
        simu_frame2[[i for i in choose_index]] = eigenframe2[[i for i in choose_index]]
        simu_frame3[[i for i in choose_index]] = eigenframe3[[i for i in choose_index]]
        simu_frame4[[i for i in choose_index]] = eigenframe4[[i for i in choose_index]]
        
        cc = eval('simu_frame'+str(_)).columns[1:]
        general_formu = cc.drop(['ADAS11', 'ADAS13', 'MMSE','APOE4'])
        try:
            general_formu = general_formu.drop('idx')
        except:
            pass
        dep_indicator = 'ADAS13'
        ps_formula = 'APOE4 ~ '+'+'.join(list(general_formu))
        rsp_formula = [dep_indicator, '~ '+'+'.join(list(general_formu)),
                       '~ '+'+'.join(list(general_formu))]
        
        part1_time+=time.perf_counter()-start
        print (time.perf_counter()-start)
        for combi in combi_lst: #【2,3,4】
            simu_frame11 = simu_frame1.copy()
            simu_frame12 = simu_frame2.copy()
            simu_frame13 = simu_frame3.copy()
            simu_frame14 = simu_frame4.copy()
            for sec_keys in ev_keys:
                if sec_keys=='ML':
                    ml_flag=True
                    dr.data_loadin(eval('simu_frame1'+str(combi[1])),eval('simu_frame1'+str(combi[0])))  
                    eval('simu_frame1'+str(combi[0]))['propensity_score'],ps_time = dr.ps_Model(ps_formula,ml_flag) 
                    ps_tm[sec_keys]+=ps_time
                    dr.data_loadin(eval('simu_frame1'+str(combi[2])),eval('simu_frame1'+str(combi[0])))
                    rsp_formu = rsp_formula[0]+rsp_formula[1]
                    eval('simu_frame1'+str(combi[0]))['u0_X'],rsp_t = dr.rsp_Model(rsp_formu,0,ml_flag,model_type='continue')   
                    rsp_tm[sec_keys]+=rsp_t
                    rsp_formu = rsp_formula[0]+rsp_formula[2]
                    eval('simu_frame1'+str(combi[0]))['u1_X'],rsp_t = dr.rsp_Model(rsp_formu,1,ml_flag,model_type='continue')   
                    rsp_tm[sec_keys]+=rsp_t
                else:
                    ml_flag=False
                    dr.data_loadin(eval('simu_frame1'+str(combi[1])),eval('simu_frame1'+str(combi[0])))
                    eval('simu_frame1'+str(combi[0]))['propensity_score'],ps_time = dr.ps_Model(ps_formula,ml_flag) 
                    ps_tm[sec_keys]+=ps_time
                    dr.data_loadin(eval('simu_frame1'+str(combi[2])),eval('simu_frame1'+str(combi[0])))
                    for label in [0,1]:
                        rsp_formu = rsp_formula[0]+rsp_formula[label+1]
                        eval('simu_frame1'+str(combi[0]))['u'+str(label)+'_X'],rsp_t = dr.rsp_Model(rsp_formu,label,ml_flag,model_type='continue')   
                        rsp_tm[sec_keys]+=rsp_t
                    
                partition_time_lst['TC_DR'][sec_keys].append(response_time+ps_time)

                data = eval('simu_frame1'+str(combi[0]))[['APOE4','propensity_score','u1_X','u0_X']] 
                data['Y_F'] = eval('simu_frame1'+str(combi[0]))[dep_indicator]
                ate_pos = (data['APOE4']*(data['Y_F']-data['u1_X']))/data['propensity_score'] + data['u1_X']
                ate_neg = ((1.0-data['APOE4'])*(data['Y_F']-data['u0_X']))/(1.0-data['propensity_score']) + data['u0_X']           
                true_ate.append(data[data['APOE4']==1]['Y_F'].mean()-data[data['APOE4']==0]['Y_F'].mean())

                TC_DR = ate_pos-ate_neg
                partition_ate['TC_DR'][sec_keys][__].append(TC_DR)

    sys.stdout.flush()
np.save('./triple-crossfitting/true/ADAS13tc_ate.npy',partition_ate)

In [ ]:
# if we got a nan value in computing the mean ATE, we can use the code below.
TC_without_nan_value = []
for i in partition:
    mean_partition = []
    for j in range(24):
        if np.isnan(partition_ate['TC_AIPW']['ML'][i][j].mean()) == False:
            mean_partition.append(partition_ate['TC_AIPW']['ML'][i][j].mean())
    TC_without_nan_value.append(np.median(mean_partition))

In [ ]:
# calculate the estimated ATE and its SD.
zb = 'MMSE'
indi = 'G_Computation'
sec_key = 'ML'
ttate = []
load_file = './triple-crossfitting/true/MMSEbaseline_ate.npy'
partition_ate = np.load(load_file,allow_pickle=True).item()
ate_p = []
for i in list(partition_ate[indi][sec_key].keys()):
    sps_p = []
    a_ate = flatten([list(i_.values) for i_ in partition_ate[indi][sec_key][i]])
    par_ate = np.mean([i for i in a_ate if np.isnan(i)==False])
    for j in range(2):
        a_sp_p = list(partition_ate['G_Computation']['ML'][i][j].values)
        sp_p = np.array([i for i in a_sp_p if np.isnan(i)==False])
        sps_p.append(np.mean(sp_p))
    single_ate = np.mean(sps_p)
    ate_p.append(single_ate)
ttate.append(ate_p)
ttate = flatten(ttate)
print (np.median(ttate))
print (np.sqrt(np.median((ttate - np.mean(ttate))**2 + np.var(ttate))))